In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from datetime import datetime

In [25]:
v10_cb = pd.read_csv("v10_cb.csv")
v10_p4 = pd.read_csv("v10_p4.csv")

In [28]:
v10_cb.shape

(7225, 2)

In [31]:
v10_cb["timestamp"].apply(lambda x: datetime.fromtimestamp(x))
# v10_cb["timestamp"] = datetime.fromtimestamp(v10_cb["timestamp"])

OSError: [Errno 22] Invalid argument

In [22]:
v10_cb["timestamp"]

0       1575790611829
1       1575790611927
2       1575790612045
3       1575790612153
4       1575790612271
            ...      
7220    1575791405634
7221    1575791405753
7222    1575791405851
7223    1575791405969
7224    1575791406067
Name: timestamp, Length: 7225, dtype: int64